In [ ]:
# %pip install numpy scipy matplotlib neuron --quiet

In [ ]:
import numpy as np
import pylab as plt
plt.ion()

In [ ]:
!nrnivmodl > null.out

# Model setting

In [ ]:
from neuron import h
soma = h.Section(name='soma')

soma.L = 10
soma.diam = 10
soma.insert('leak')
soma.gl_leak = 0.0003

## Define inputs

In [ ]:
# step current
iclamp = h.IClamp(soma(0.5))

tstop = 100
onset = 20
dur = tstop - 2 * onset

iclamp.delay = onset
iclamp.dur = dur
iclamp.amp = 0.008 # (nA)

# record voltage and time
v = h.Vector().record(soma(0.5)._ref_v)             # Membrane potential vector
t = h.Vector().record(h._ref_t)                     # Time stamp vector

# Run simulation

In [ ]:
h.load_file('stdrun.hoc')
h.finitialize(-65)
h.tstop = tstop
h.run()

In [ ]:
v_arr = np.array(v)
t_arr = np.array(t)

plt.figure()
plt.plot(t_arr, v_arr)
plt.xlim(onset-10, (onset+dur)+10)
plt.xlabel('t (ms)')
plt.ylabel('v (mV)')
plt.grid()

# indexes between onset and onset+dur
idx_t = np.where((t_arr>onset)&(t_arr<(onset+dur)))[0]
plt.ylim(v_arr[idx_t].min()-0.25,v_arr[idx_t].max()+0.25)

print("minimum %g and maximum voltage %g (mV) on the [%g,%g] (ms) time interval" % (v_arr[idx_t].min(), v_arr[idx_t].max(), onset, onset+dur))
print("Delta V is %g (mV)" % (v_arr[idx_t].max()-v_arr[idx_t].min()))

# Passive properties of the neuron from fitting

Now we want to fit the rising phase from *onset* to *onset+dur*

In [ ]:
from scipy.optimize import curve_fit
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

def func_exp_grow(x, v0, delta_v, x0, tau):
    """Decaying exponential function."""
    return v0 + delta_v * (1-np.exp(-(x-x0)/tau))

# select x (time) on which to perform fit
x = t_arr[idx_t]
y_sim = v_arr[idx_t]

popt, _ = curve_fit(func_exp_grow, x, y_sim, p0=[-54, 5, 20, 10], bounds=([-100, 0, 19.99, 0.001], [100, 100, 20.01, 100]))

In [ ]:
popt

In [ ]:
y_fit = func_exp_grow(x,*popt)

In [ ]:
plt.plot(x, y_sim, 'ko', markersize=2)
plt.plot(x, y_fit, 'r-')

print("absolute error = %g " % np.mean((y_sim-y_fit)**2))

In [ ]:
tau_cut_off = soma.cm * (1/soma.gl_leak) * 1e-6 * 1e3  # (uF/cm2) / (mS/cm2)
print("The time constant is %2.3g (ms)" % tau_cut_off)

# Compute input resistance

Here you will have to reproduce the simultation above for different input currents and record the Delta V

In [ ]:
# 
icc = np.array([0.002, 0.004, 0.006, 0.008]) # input currents go here !
dv = np.array([2.15445477, 4.28288189, 6.39247078, 8.51333235]) # Delta V go here !

In [ ]:
def func_lin(x, q, m):
    """Linear fit"""
    return q + m * x

popt_lin, _ = curve_fit(func_lin, icc, dv)

Rin = popt_lin[1] * 1e6
print('input resistance from fit = %g (Ohm)' % Rin)

plt.plot(icc, dv,'ko')
plt.plot(icc, func_lin(icc,*popt_lin),'r-');

In [ ]:
surface = soma.L * soma.diam * np.pi # um2 #  = soma(0.5).area() 
gleakage = surface * soma.gl_leak * 1e-8 # um^2*S/cm^2 = 10^-12/10^-4 = 10^-8
Rleak = ( 1 / gleakage ) / 1e9
print('Input resistance of the model = %g (GOhm)' % Rleak)